# Projeto Final: Neural Machine Translation

Aluno: Leonam Rezende Soares de Miranda

Nesse projeto será construido um modelo de tradução automática neural de inglês para alemão usando LSTMs com attention. A tradução automática é uma tarefa importante no processamento de linguagem natural e pode ser útil não apenas para traduzir um idioma para outro, mas também para desambiguar o sentido das palavras (por exemplo, determinar se a palavra "banco" se refere ao banco financeiro ou à terra ao longo de um rio). Implementar isso usando apenas uma Rede Neural Recorrente (RNN) com LSTMs pode funcionar para sentenças curtas a médias, mas pode resultar no problema do desaparecimento do gradientes para sequências muito longas. Para resolver isso, seá utilizado o mecanismo de mecanismo de atenção para permitir que o decodificador acesse todas as partes relevantes da frase de entrada, independentemente de seu comprimento.


## Sumários
- [Parte 1: Preparação dos Dados](#1)
    - [1.1  Importação dos Dado](#1.1)
    - [1.2  Tokenização e Formatação](#1.2)
    - [1.3  Funções Auxiliares: tokenize e detokenize](#1.3)
    - [1.4  Bucketing](#1.4)
    - [1.5  Exploração dos Dados](#1.5)
- [Part 2:  Tradução Automática Neural com Atenção](#2)
    - [2.1  Attention Visão Geral](#2.1)
    - [2.2  Funções auxiliares](#2.2)
    - [2.3  Implementação - Visão Geral](#2.3)
- [Part 3:  Trainamento](#3)
    - [3.1  TrainTask](#3.1)
    - [3.2  EvalTask](#3.2)
    - [3.3  Loop](#3.3)
- [Part 4:  Teste](#4)
    - [4.1  Decoding](#4.1)
    - [4.2  Minimum Bayes-Risk Decoding](#4.2)

<a name="1"></a>
# Parte 1:  Preparação dos Dados


Primeiramente serão importandos os pacotes que serão usado neste trabalho. Será utilizado o framework Trax criada e mantida pela equipe do Google Brain team para fazer a maior parte do trabalho pesado. Este framework submódulos para buscar e processar os conjuntos de dados, bem como construir e treinar o modelo.

In [16]:
from termcolor import colored
import random
import numpy as np

import trax
from trax import layers as tl
from trax.fastmath import numpy as fastnp
from trax.supervised import training

!pip list | grep trax

trax                     1.3.4
You should consider upgrading via the '/opt/conda/bin/python3 -m pip install --upgrade pip' command.


A seguir, importaremos o conjunto de dados que usaremos para treinar o modelo. Para atender às restrições de armazenamento neste ambiente de laboratório, usaremos apenas um pequeno conjunto de dados da [Opus](http://opus.nlpl.eu/), uma coleção crescente de textos traduzidos da web. Em particular, obteremos um subconjunto de tradução do inglês para o alemão especificado como `opus / medical`, que contém textos relacionados à medicina. Se o armazenamento não for um problema, você pode optar por obter um corpus maior, como o conjunto de dados de tradução de inglês para alemão de [ParaCrawl](https://paracrawl.eu/), um grande conjunto de dados de tradução multilíngue criado pela União Europeia. Ambos os conjuntos de dados estão disponíveis via [Tensorflow Datasets (TFDS)](https://www.tensorflow.org/datasets) e pode-se navegar pelos outros conjuntos de dados disponíveis [aqui](https://www.tensorflow.org/datasets/catalog/overview). Baixamos os dados para você no diretório '/data/' do seu espaço de trabalho. Como você verá abaixo, você pode acessar facilmente este conjunto de dados do TFDS com `trax.data.TFDS`. O resultado é uma função de gerador python que produz tuplas. Use o argumento `keys` para selecionar o que aparece em qual posição na tupla. Por exemplo, `keys = ('en', 'de')` abaixo retornará pares como (frase em inglês, frase em alemão). 

In [27]:
# Obtem a função de geradora para o conjunto de treinamento
# Isso fará o download do conjunto de treinamento se nenhum data_dir for especificado.
train_stream_fn = trax.data.TFDS('opus/medical',
                                 data_dir='./data/',
                                 keys=('en', 'de'),
                                 eval_holdout_size=0.01, # 1% for eval
                                 train=True)

# Obtem a função de geradora para o conjunto de avaliação
eval_stream_fn = trax.data.TFDS('opus/medical',
                                data_dir='./data/',
                                keys=('en', 'de'),
                                eval_holdout_size=0.01, # 1% for eval
                                train=False)

Observe que TFDS retorna uma *função geradora*, não um gerador. Isso ocorre porque, em Python, você não pode redefinir geradores, portanto, não pode voltar a um valor gerado anteriormente. Durante o treinamento de aprendizado profundo, você usa-se Stochastic Gradient Descent e não precisa realmente de voltar - mas às vezes é bom ser capaz de fazer isso, e é aí que as funções entram. Na verdade, é muito comum usar funções geradoras em Python - por exemplo, `zip` é uma função geradora. Para saber mais sobre [geradores Python](https://book.pythontips.com/en/latest/generators.html). A seguir, será impresso um par de amostras dos dados de treinamento e validação. Observe que a saída bruta é representada em bytes (denotados pelo prefixo 'b') e estes serão convertidos em strings internamente nas próximas etapas.

In [28]:
train_stream = train_stream_fn()
print(colored('train data (en, de) tuple:', 'red'), next(train_stream))
print()

eval_stream = eval_stream_fn()
print(colored('eval data (en, de) tuple:', 'red'), next(eval_stream))

train data (en, de) tuple: (b'During treatment with olanzapine, adolescents gained significantly more weight compared with adults.\n', b'W\xc3\xa4hrend der Behandlung mit Olanzapin nahmen die Jugendlichen im Vergleich zu Erwachsenen signifikant mehr Gewicht zu.\n')

eval data (en, de) tuple: (b'Lutropin alfa Subcutaneous use.\n', b'Pulver zur Injektion Lutropin alfa Subkutane Anwendung\n')


<a name="1.2"></a>
## 1.2  Tokenização e Formatação

Agora que importamos nosso corpus, iremos pré-processar as sentenças em um formato que nosso modelo possa aceitar. Isso será composto de várias etapas:

**Tokenizar as frases usando representações de subpalavra:**  queremos representar cada frase como um array de inteiros em vez de strings. Para nossa aplicação, usaremos representações de *subpalavra* para tokenizar as sentenças. Esta é uma técnica comum para evitar palavras fora do vocabulário, permitindo que partes das palavras sejam representadas separadamente. Por exemplo, em vez de ter entradas separadas em seu vocabulário para --"fear", "fearless", "fearsome", "some", and "less"--, pode-se simplesmente armazenar --"fear", "some", and "less"-- em seguida, permita que o tokenizador combine essas sub-palavras quando necessário. Isso permite que o modelo seja mais flexível de forma que não precise salvar palavras incomuns explicitamente em seu vocabulário. A tokenização é feita com o comando 'trax.data.Tokenize ()' e foi bixado o vocabulário de subpalavra combinado para inglês e alemão (ou seja, `ende_32k.subword`) salvo no diretório` data`. Este vocabulário de subpalavras foi baixado do curso de NLP fornecido pelo [Coursera](https://www.coursera.org/specializations/natural-language-processing).

In [29]:
# variáveis globais que indicam o nome do arquivo e o diretório do arquivo de vocabulário
VOCAB_FILE = 'ende_32k.subword'
VOCAB_DIR = 'data/'

# Tokenize o conjunto de dados.
tokenized_train_stream = trax.data.Tokenize(vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR)(train_stream)
tokenized_eval_stream = trax.data.Tokenize(vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR)(eval_stream)

**Anexe um token de final de frase a cada frase:** Será atribuído um token (ou seja, neste caso, o número **1**) para marcar o final de uma frase. Isso será útil em inferência/previsão, portanto, saberemos que o modelo concluiu a tradução.

In [30]:
# Anexe EOS no final de cada frase.

# Número inteiro atribuído como final de frase (EOS)
EOS = 1

# função auxiliar do gerador para anexar EOS a cada frase
def append_eos(stream):
    for (inputs, targets) in stream:
        inputs_with_eos = list(inputs) + [EOS]
        targets_with_eos = list(targets) + [EOS]
        yield np.array(inputs_with_eos), np.array(targets_with_eos)

# anexa EOS aos dados de treinamento
tokenized_train_stream = append_eos(tokenized_train_stream)

# anexa EOS aos dados de validação
tokenized_eval_stream = append_eos(tokenized_eval_stream)

**Filtrar frases longas:** Será imposto um limite no número de tokens por frase para garantir que não fiquemos sem memória. Isso é feito com o método `trax.data.FilterByLength ()` e você pode ver sua sintaxe abaixo.

In [36]:
# Filtre frases muito longas para não ficar sem memória.
# length_keys = [0, 1] significa que filtramos frases em inglês e alemão, então
# ambos não devem ser mais longos que 256 tokens para treinamento e 512 para eval.
filtered_train_stream = trax.data.FilterByLength(
    max_length=256, length_keys=[0, 1])(tokenized_train_stream)
filtered_eval_stream = trax.data.FilterByLength(
    max_length=512, length_keys=[0, 1])(tokenized_eval_stream)

# imprimir um exemplo de par de entrada-destino de sentenças tokenizadas
train_input, train_target = next(filtered_train_stream)
print(colored(f'Single tokenized example input:', 'red' ), train_input)
print(colored(f'Single tokenized example target:', 'red'), train_target)

Single tokenized example input: [ 2937    48   604     4  2960    15  7797 12088   473     9     4 20216
    70 13814  8689  1389     7   139  4773 16766 22939    76    66    13
 20216    70  8781     5  3550 30650  4729   992     1]
Single tokenized example target: [25593     5    67    20 14297 12088    44     6    10  1487    38  8542
 10079  9567    14    97   225 21417  8863  6859  7347  3550 30650  4729
   992     1]


<a name="1.3"></a>
## 1.3  Funções Auxiliares: tokenize e detokenize

Dado qualquer conjunto de dados, devemos ser capazes de mapear palavras para seus índices e índices para suas palavras. As entradas e saídas dos modelos trax são geralmente tensores de números, onde cada número corresponde a uma palavra. Se fossemos processar os dados manualmente, teriamos que usar o seguinte:

- <span style='color:blue'> word2Ind: </span> a dictionary mapping the word to its index.
- <span style='color:blue'> ind2Word:</span> a dictionary mapping the index to its word.
- <span style='color:blue'> word2Count:</span> a dictionary mapping the word to the number of times it appears. 
- <span style='color:blue'> num_words:</span> total number of words that have appeared. 

Ao utilizar trax, não são utilizar essas estruturas de dados para mapeamento dos dados (pois são encapsuladas). A seguir foram criadas duas funções auxiliares para lidar com os dados: 

- `tokenize()`: converte uma frase de texto em sua lista de tokens correspondente (ou seja, lista de índices). Também converte palavras em subpalavras (partes de palavras).
- `detokenize()`: converte uma lista de tokens em sua sentença correspondente (ou seja, string).

In [37]:
def tokenize(input_str, vocab_file=None, vocab_dir=None):
    """Codifica uma sentença num vetor de inteiros

    Args:
        input_str (str): frase legível por humanos para codificar
        vocab_file (str): nome do arquivo do vocabulário
        vocab_dir (str): caminho do arquivo do vocabulário
  
    Retornos:
        numpy.ndarray: versão tokenizada da sentença de entrada
    """
    
    # Define a codificação do "final da frase" como 1
    EOS = 1
    
    # Use o método trax.data.tokenize. Recebe streams e retorna streams,
    # contornado isso criando um fluxo de 1 elemento com `iter`.
    inputs =  next(trax.data.tokenize(iter([input_str]),
                                      vocab_file=vocab_file, vocab_dir=vocab_dir))
    
    # Marca o fim de sentença com EOS
    inputs = list(inputs) + [EOS]
    
    # Adding the batch dimension to the front of the shape
    batch_inputs = np.reshape(np.array(inputs), [1, -1])
    
    return batch_inputs


def detokenize(integers, vocab_file=None, vocab_dir=None):
    """Decodifica um veotr de inteiros para uma sentença

    Args:
        integers (numpy.ndarray): vetor de inteiros para decodificar
        vocab_file (str): nome do arquivo do vocabulário
        vocab_dir (str): caminho do arquivo do vocabulário
  
    Retornos:
        str: a sentença decodificada.
    """
    
    # Remove as dimensões de tamanho 1
    integers = list(np.squeeze(integers))
    
    # Set the encoding of the "end of sentence" as 1
    EOS = 1
    
    # Remova o EOS para decodificar apenas os tokens originais
    if EOS in integers:
        integers = integers[:integers.index(EOS)] 
    
    return trax.data.detokenize(integers, vocab_file=vocab_file, vocab_dir=vocab_dir)

A seguir veremos como utilizar essas funções:

In [39]:
# Detokenize um par de entrada-destino de sentenças tokenizadas
print(colored(f'Single detokenized example input:', 'red'), detokenize(train_input, vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR))
print(colored(f'Single detokenized example target:', 'red'), detokenize(train_target, vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR))
print()

# Tokenize e detokenize uma palavra que não é explicitamente salva no arquivo de vocabulário.
# Veja como são combinadas as subpalavras -- 'hell' and 'o'-- para formar a palavra 'hello'.
print(colored(f"tokenize('hello'): ", 'green'), tokenize('hello', vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR))

Single detokenized example input: Do not put the pre-filled pen next to the freezer compartment of your refrigerator or a freezer pack.

Single detokenized example target: Legen Sie den Fertigpen nicht in die Nähe des Gefrierfachs oder eines Kühlelements.


tokenize('hello'):  [[17332   140     1]]


<a name="1.4"></a>
## 1.4  Bucketing

Bucketing as frases tokenizadas é uma técnica importante usada para acelerar o treinamento em PNL.
Aqui está um [bom artigo que o descreve em detalhes](https://medium.com/@rashmi.margani/how-to-speed-up-the-training-of-the-sequence-model-using-bucketing-techniques- 9e302b0fd976)
mas essencialmente é algo muito simples. As entradas têm comprimentos variáveis e você deseja torná-los iguais ao agrupar grupos de frases em lote. Uma maneira de fazer isso é realizar padding com cada frase com o comprimento da frase mais longa do conjunto de dados. Isso pode levar a algum desperdício de computação. Por exemplo, se houver várias frases curtas com apenas dois tokens, queremos preenchê-los quando a frase mais longa for composta de 100 tokens? Em vez de preencher com 0s até o comprimento máximo de uma frase a cada vez, podemos agrupar as frases tokenizadas por comprimento e intervalo, como nesta imagem (do artigo acima):

![alt text](https://miro.medium.com/max/700/1*hcGuja_d5Z_rFcgwe9dPow.png)

Agrupa-se sentenças de tamanho similar juntas e assim, adiciona-se um padding mínimo para tornar as sentenças num mesmo batch de comprimento igual (usualmente a potência de 2 mais próxima). Isso permite evitar disperdicios computacionais ao processar sequências com padding.
No Trax, o Bucketing é implementado na função [bucket_by_length](https://github.com/google/trax/blob/5fb8aa8c5cb86dabb2338938c745996d5d87d996/trax/supervised/inputs.py#L378).

In [40]:
# Bucketing to create streams of batches.

# Os Buckets são definidos em termos de limites e tamanhos de batch
# Batch_sizes[i] determina o tamanho de batch para itens com length < boundaries[i]
# So below, we'll take a batch of 256 sentences of length < 8, 128 if length is
# Então, abaixo, pegaremos um lote de 256 sentenças de comprimento <8, 128 se o comprimento for
# entre 8 e 16 e assim por diante - e apenas 2 se o comprimento for superior a 512.

boundaries =  [8,   16,  32, 64, 128, 256, 512]
batch_sizes = [256, 128, 64, 32, 16,    8,   4,  2]

# Cria os geradores
train_batch_stream = trax.data.BucketByLength(
    boundaries, batch_sizes,
    length_keys=[0, 1]  # Como antes: conte as entradas e alvos até o comprimento.
)(filtered_train_stream)

eval_batch_stream = trax.data.BucketByLength(
    boundaries, batch_sizes,
    length_keys=[0, 1]  # Como antes: conte as entradas e alvos até o comprimento.
)(filtered_eval_stream)

# Adiciona mascara para os paddings (0s)
train_batch_stream = trax.data.AddLossWeights(id_to_mask=0)(train_batch_stream)
eval_batch_stream = trax.data.AddLossWeights(id_to_mask=0)(eval_batch_stream)

<a name="1.5"></a>
## 1.5  Exploração dos Dados

Agora exibiremos alguns de nossos dados. Vamos primeiro obter o gerador de dados e obter um lote de dados

In [41]:
input_batch, target_batch, mask_batch = next(train_batch_stream)

# vamos ver o tipo de dados de um batch
print("input_batch data type: ", type(input_batch))
print("target_batch data type: ", type(target_batch))

# vamos ver a forma deste batch específico (comprimento do batch, comprimento da frase)
print("input_batch shape: ", input_batch.shape)
print("target_batch shape: ", target_batch.shape)

input_batch data type:  <class 'numpy.ndarray'>
target_batch data type:  <class 'numpy.ndarray'>
input_batch shape:  (32, 64)
target_batch shape:  (32, 64)


O `input_batch` e` target_batch` são vetores Numpy que consistem em sentenças tokenizadas em inglês e em alemão, respectivamente. Esses tokens serão usados posteriormente para produzir vetores de embedding para cada palavra na frase (portanto, o embedding de uma frase será uma matriz). O número de frases em cada lote é geralmente uma potência de 2 para o uso ideal da memória do computador.

Agora podemos inspecionar visualmente alguns dos dados. Apenas para observar, embora este seja um conjunto de dados padrão amplamente usado, ele possui algumas traduções incorretas conhecidas. Com isso, vamos escolher uma frase aleatória e imprimir sua representação tokenizada.

In [65]:
# escolhe um índice aleatório menor que o tamanho do batch.
index = random.randrange(len(input_batch))

# use o índice para obter uma entrada do lote de entrada e destino
print(colored('ESTA É A SENTENÇA EM INGLÊS: \n', 'red'), detokenize(input_batch[index], vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR), '\n')
print(colored('ESTA É A VERSÃO TOKENIZADA DA SENTENÇA EM INGLÊS: \n ', 'red'), input_batch[index], '\n')
print(colored('ESTA É A SENTENÇA EM ALEMÃO: \n', 'red'), detokenize(target_batch[index], vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR), '\n')
print(colored('ESTA É A VERSÃO TOKENIZADA DA SENTENÇA EM ALEMÃO: \n', 'red'), target_batch[index], '\n')

ESTA É A SENTENÇA EM INGLÊS: 
 In the GIST clinical trial, both gastrointestinal and intra-tumoural haemorrhages were reported (see section 4.8).
 

ESTA É A VERSÃO TOKENIZADA DA SENTENÇA EM INGLÊS: 
  [   71     4  9168 21165 15481  8686     2   298 15256 17233 27307   215
     8 19978     5    15  9996  5501   278  5802  6165 19662 23779    33
   152  4981    50   372  2745   219     3   363 33022 30650  4729   992
     1     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0] 

ESTA É A SENTENÇA EM ALEMÃO: 
 In der klinischen GIST-Studie wurden sowohl gastrointestinale als auch intratumorale Blutungen beobachtet (siehe Abschnitt 4.8).
 

ESTA É A VERSÃO TOKENIZADA DA SENTENÇA EM ALEMÃO: 
 [   71    11 28609  9168 21165    15  5355   302   957 15256 17233 27307
   347    69    74 19978  9996 28091    35  9208   277 17791     5    50
  4588 10729   219     3   363 33022 30650  

<a name="2"></a>
# Parte 2:  Neural Machine Translation with Attention

Agora que temos os geradores de dados e realizou o pré-processamento, é hora de construir o modelo.

<a name="2.1"></a>
## 2.1  Attention Visão Geral

O modelo que será construidp usa uma arquitetura de codificador-decodificador. Essa Rede Neural Recorrente (RNN) receberá uma versão tokenizada de uma frase em seu codificador e, em seguida, a passará para o decodificador para tradução. Entretanto, apenas de um modelo de sequência a sequência regular com LSTMs funcionar efetivamente para frases curtas a médias, mas performance começará a degradar para frases mais longas. Esse problema pode ser vizualizado na figura abaixo, onde todo o contexto da sentença de entrada é compactado em um vetor que é passado para o bloco do decodificador. É visível que isso será um problema para frases muito longas (por exemplo, 100 tokens ou mais) porque o contexto das primeiras partes da entrada terá muito pouco efeito no vetor final passado para o decodificador.



![alt text](https://drive.google.com/uc?id=10QcWt6IgSH5LpeD14Z8_HmUhOTJsN2Ax)

Adicionando uma camada de atenção a este modelo evita esse problema, dando ao decodificador acesso a todas as partes da frase de entrada. Para ilustrar, vamos apenas usar uma frase de entrada de 4 palavras, conforme mostrado abaixo. Lembre-se de que um estado oculto é produzido a cada passo de tempo do codificador (representado pelos retângulos laranja). Todos eles são passados ​​para a camada de atenção e cada um recebe uma pontuação de acordo com a ativação atual (ou seja, estado oculto) do decodificador. Por exemplo, vamos considerar a figura abaixo onde a primeira previsão "Wie" já foi feita. Para produzir a próxima previsão, a camada de atenção receberá primeiro todos os estados ocultos do codificador (ou seja, retângulos laranja), bem como o estado oculto do decodificador ao produzir a palavra "Wie" (ou seja, primeiro retângulo verde). Dadas essas informações, ele pontuará cada um dos estados ocultos do codificador para saber em qual deles o decodificador deve se concentrar para produzir a próxima palavra. O resultado do treinamento do modelo pode ter aprendido que ele deve se alinhar ao segundo estado oculto do codificador e, subsequentemente, atribuir uma alta probabilidade à palavra "geht". Se estivermos usando a decodificação gananciosa, produziremos a palavra dita como o próximo símbolo e, em seguida, reiniciaremos o processo para produzir a próxima palavra até chegarmos a uma previsão do final da frase.

![alt text](https://drive.google.com/uc?id=10X_jMhFFg7tJZZWmrJL6-3xNouDtY4Sz)

Existem diferentes maneiras de implementar a atenção e a que usaremos para esta tarefa é a Scaled Dot Product Attention, que tem o formato:

$$Attention(Q, K, V) = softmax(\frac{QK^T}{\sqrt{d_k}})V$$

Essa equação pode ser entendida como pontuações computacionais usando consultas (Q) e chaves (K), seguidas por uma multiplicação de valores (V) para obter um vetor de contexto em um determinado passo de tempo do decodificador.

<a name="2.2"></a>
## 2.2  Funções Auxiliares

Implementaremos primeiro algumas funções que usaremos mais tarde. Serão funções utilizadas noo codificador de entrada, decodificador de pré-atenção e preparação das consultas, chaves, valores e máscara.

### 2.2.1 Codificador de Entrada

O codificador de entrada é executado nos tokens de entrada, cria seus embeddings e os alimenta em uma rede LSTM. Isso gera as ativações que serão as chaves e os valores de atenção. É uma rede [Serial](https://trax-ml.readthedocs.io/en/latest/trax.layers.html#trax.layers.combinators.Serial) que usa: 

   - [tl.Embedding](https://trax-ml.readthedocs.io/en/latest/trax.layers.html#trax.layers.core.Embedding): Converte cada token em sua representação vetorial. Neste caso, é o tamanho do vocabulário pela dimensão do modelo: `tl.Embedding (vocab_size, d_model)`. `vocab_size` é o número de entradas no vocabulário fornecido. `d_model` é o número de elementos do embbeding.
  
   - [tl.LSTM](https://trax-ml.readthedocs.io/en/latest/trax.layers.html#trax.layers.rnn.LSTM): LSTM camada de tamanho `d_model`. Queremos ser capazes de configurar quantas camadas de codificador temos, então serão criadas camadas LSTM iguais ao número do parâmetro `n_encoder_layers`.
   
<img src = "https://drive.google.com/uc?id=1yv3DaPPOxo-ADIdZn4yeUiJXnoh0gIJk">

In [43]:
def input_encoder_fn(input_vocab_size, d_model, n_encoder_layers):
    """ O codificador de entrada é executado na sentença de entrada e 
        cria ativações que serão as chaves e os valores para a camada de atenção
    
    Args:
        input_vocab_size: int: tamanho do vocabulário da entrada
        d_model: int:  profundidade do embbeding (n_units in the LSTM cell)
        n_encoder_layers: int: number of LSTM layers in the encoder
    Retornos:
        tl.Serial: The input encoder
    """
    
    # cria uma rede em série
    input_encoder = tl.Serial( 
        
        # camada de embedding para converter tokens em vetores
        tl.Embedding(vocab_size=input_vocab_size, d_feature=d_model),
        
        # Alimente as camadas LSTMs com os embeddings. 
        # É uma pilha de n_encoder_layers camadas LSTM
        [tl.LSTM(n_units=d_model) for _ in range(n_encoder_layers)]

    )

    return input_encoder

### 2.2.2 Pre-attention decoder

O decodificador de pré-atenção é executado nos alvos e cria ativações que são usadas como queries na camada de atenção. Esta é uma rede serial composta por:

   - [tl.ShiftRight](https://trax-ml.readthedocs.io/en/latest/trax.layers.html#trax.layers.attention.ShiftRight): Essa camada realiza padding de um token no início dos tokens alvos  (E.x.: `[8, 34, 12]` deslocado para a direita é `[0, 8, 34, 12]`). Isso funcionará como um token de início de frase que será a primeira entrada para o decodificador. Durante o treinamento, essa mudança também permite que os tokens de destino sejam passados como entrada para fazer *teacher forcing*, comparando a predição com a saída esperada durante o treinamento, resultando num treinamento mais rápido e com maior acurácia.

   - [tl.Embedding](https://trax-ml.readthedocs.io/en/latest/trax.layers.html#trax.layers.core.Embedding): Essa camadaconverte cada token em sua representação vetorial. Neste caso, é o tamanho do vocabulário pela dimensão do modelo: `tl.Embedding (vocab_size, d_model)`. `vocab_size` é o número de entradas no vocabulário fornecido. `d_model` é o número de elementos na palavra embedding.
   
   - [tl.LSTM](https://trax-ml.readthedocs.io/en/latest/trax.layers.html#trax.layers.rnn.LSTM): LSTM layer of size `d_model`.

<img src = "https://drive.google.com/uc?id=14JKWdVedr3meHB_BByQkZQF_XO0aqCpf">

In [44]:
def pre_attention_decoder_fn(mode, target_vocab_size, d_model):
    """ O decodificador de pré-atenção é executado nos alvos e cria ativações
     que são usadas como queries na cada de atenção.
    
    Args:
        mode: str: 'train' or 'eval'
        target_vocab_size: int: tamanho do vocabulário do alvo (target)
        d_model: int:  profundidade do embedding (n_units na celula LSTM)
    Retornos:
        tl.Serial: O decodificador de pré-atenção
    """
    
    # cria uma rede em série
    pre_attention_decoder = tl.Serial(
        
        # desloque para a direita para inserir token de início de frase 
        # e implementar teacher forcing during training
        tl.ShiftRight(mode=mode),

        # camada de embedding para converter tokens em vetores
        tl.Embedding(vocab_size=target_vocab_size, d_feature=d_model),

        # Somente uma camada LSTM com d_model celulas
        tl.LSTM(n_units=d_model)
    )
    
    return pre_attention_decoder

### 2.2.3 Preparando a entrada da camada de atenção

Esta função irá preparar as entradas para a camada de atenção. Queremos pegar as ativações do codificador e do decodificador de pré-atenção e atribuí-las às queries, chaves e valores. Além disso, outra saída aqui será a máscara para distinguir tokens reais de tokens de preenchimento. Essa máscara será usada internamente pelo Trax ao calcular o softmax para que os tokens de preenchimento não tenham efeito nas probabilidades calculadas. A partir das etapas de preparação de dados realizadas na Seção 1, é possível saber quais tokens na entrada correspondem ao preenchimento.

As últimas 2 linhas estão relacionadas a *multiheaded attention* o que pode ser entendido como calcular attention multiplas vezes para melhorar as predições do modelo. O importante agora é  aber quais devem ser as queries, chaves e valores, bem como inicializar a máscara.


In [48]:
def prepare_attention_input(encoder_activations, decoder_activations, inputs):
    """Prepara queries, keys, values and mask para a camada de atenção.
    
    Args:
        encoder_activations fastnp.array(batch_size, padded_input_length, d_model): saída do codificador da entrada
        decoder_activations fastnp.array(batch_size, padded_input_length, d_model): saída do decodificador de pré-atenção
        inputs fastnp.array(batch_size, padded_input_length): tokens de entrada com padding
    
    Retornos:
        queries, keys, values and mask for attention.
    """
    # set the keys and values to the encoder activations
    keys = encoder_activations
    values = encoder_activations
    
    # set the queries to the decoder activations
    queries = decoder_activations
    
    # generate the mask to distinguish real tokens from padding
    # hint: inputs is 1 for real tokens and 0 where they are padding
    mask = inputs != 0
    
    # add axes to the mask for attention heads and decoder length.
    mask = fastnp.reshape(mask, (mask.shape[0], 1, 1, mask.shape[1]))
    
    # broadcast so mask shape is [batch size, attention heads, decoder-len, encoder-len].
    # note: for this assignment, attention heads is set to 1.
    mask = mask + fastnp.zeros((1, 1, decoder_activations.shape[1], 1))
        
    
    return queries, keys, values, mask

<a name="2.3"></a>
## 2.3  Implementação - Visão Geral

Agora estamos prontos para implementar nosso modelo de sequência a sequência com atenção. Esta será uma rede serial e é ilustrada no diagrama abaixo. Ele mostra as camadas que você usará no Trax e você verá que cada etapa pode ser implementada facilmente com comandos de uma linha.

<img src = "https://drive.google.com/uc?id=17daxMohCkR3UzsG42dTtxFi2QKkEip_O">

Agora será implementado a função `NMTAttn` que define o modelo trax de tradução de máquina que utiliza atenção. Para isso serão seguidos os seguintes passos:

**Passo 0:** Prepara o codificador da entrada e o decodificador de pré-atenção. Essas função já foram definidas anteriormente como funções auxiliares, então é apenas uma questão de chamar essas funções e atribuí-las a variáveis.

**Passo 1:** Crie uma rede serial. Isso empilhará as camadas nas próximas etapas, uma após a outra. [tl.Serial](https://trax-ml.readthedocs.io/en/latest/trax.layers.html#trax.layers.combinators.Serial). 

**Passo 2:** Faça uma cópia dos tokens de entrada e de destino. Como você vê no diagrama acima, os tokens de entrada e de destino serão alimentados em diferentes camadas do modelo. Será utilizado a camada [tl.Select](https://trax-ml.readthedocs.io/en/latest/trax.layers.html#trax.layers.combinators.Select) para criar copias desses tokens. Eles serão odenados da seguinte forma: `[input tokens, target tokens, input tokens, target tokens]`.

**Passo 3:** Crie caminhos em paralelo para alimentar os tokens de entrada para o `input_encoder` e os tokens de destino para o` pre_attention_decoder`. Será utilizado [tl.Parallel](https://trax-ml.readthedocs.io/en/latest/trax.layers.html#trax.layers.combinators.Parallel) to create these sublayers in parallel.

**Passo 4:** Em seguida, chame a função `prepare_attention_input` para converter o codificador e as ativações do decodificador de pré-atenção para um formato que a camada de atenção aceite. Será utilizado [tl.Fn](https://trax-ml.readthedocs.io/en/latest/trax.layers.html#trax.layers.base.Fn) para trealizar essa chamada.

**Passo 5:** Agora será alimentada com (queries, keys, values, and mask) a camada de atenção [tl.AttentionQKV](https://trax-ml.readthedocs.io/en/latest/trax.layers.html#trax.layers.attention.AttentionQKV). Será calculado o "scaled dot product attention", coforme a equação mostrada na introdução dessa seção, tendo como saída os pesos de atenção (attention weights) e a máscara (mask). 
Observe que, essa camada é, na verdade, composta por uma rede profunda composta por vários ramos. Mostraremos a implementação realizada [aqui] (https://github.com/google/trax/blob/master/trax/layers/attention.py#L61) para ver as diferentes camadas usadas.

```python
def AttentionQKV(d_feature, n_heads=1, dropout=0.0, mode='train'):
  """Returns a layer that maps (q, k, v, mask) to (activations, mask).

  See `Attention` above for further context/details.

  Args:
    d_feature: Depth/dimensionality of feature embedding.
    n_heads: Number of attention heads.
    dropout: Probababilistic rate for internal dropout applied to attention
        activations (based on query-key pairs) before dotting them with values.
    mode: Either 'train' or 'eval'.
  """
  return cb.Serial(
      cb.Parallel(
          core.Dense(d_feature),
          core.Dense(d_feature),
          core.Dense(d_feature),
      ),
      PureAttention(  # pylint: disable=no-value-for-parameter
          n_heads=n_heads, dropout=dropout, mode=mode),
      core.Dense(d_feature),
  )
```

Por ter camadas profundas há o risco de desaparecimento de gradientes durante o treinamento e gostaríamos de atenuar isso. Para melhorar a capacidade de aprendizagem da rede, podemos inserir uma camada [tl.Residual](https://trax-ml.readthedocs.io/en/latest/trax.layers.html#trax.layers.combinators.Residual) para adicionar a saída de AttentionQKV com a entrada `queries`. Isso pode ser feito no trax simplesmente aninhando a camada `Atenção QKV` dentro da camada` Residual`. A biblioteca se encarregará de ramificar e adicionar para você.

**Passo 6:** Não precisaremos da máscara para o modelo que estamos construindo, então pode-se descartá-la com segurança. Neste ponto na rede, a pilha de sinal atualmente tem `[ativação de atenção, máscara, tokens de destino]` e será usada [tl.Select](https://trax-ml.readthedocs.io/en/latest/trax .layers.html # trax.layers.combinators.Select) para reduzir a pilha para apenas `[ativação de atenção, tokens de destino]`

**Passo 7:** Agora podemos alimentar a saída ponderada de atenção para o decodificador LSTM. Podemos empilhar várias camadas [tl.LSTM](https://trax-ml.readthedocs.io/en/latest/trax.layers.html#trax.layers.rnn.LSTM) para melhorar a saída, definido pelo parâmetro `n_decoder_layers`.

**Passo 8:** Queremos determinar as probabilidades de cada subpalavra no vocabulário e você pode configurar isso facilmente com uma camada [tl.Dense](https://trax-ml.readthedocs.io/en/latest/trax.layers.html#trax.layers.core.Dense), tornando seu tamanho igual ao tamanho do nosso vocabulário.

**Passo 9:** Normalize a saída para registrar as probabilidades passando as ativações na Etapa 8 para uma camada [tl.LogSoftmax](https://trax-ml.readthedocs.io/en/latest/trax.layers.html#trax.layers.core.LogSoftmax ).

In [49]:
def NMTAttn(input_vocab_size=33300,
            target_vocab_size=33300,
            d_model=1024,
            n_encoder_layers=2,
            n_decoder_layers=2,
            n_attention_heads=4,
            attention_dropout=0.0,
            mode='train'):
    """Retorna um modelo de sequência a sequência LSTM com atenção.

    A entrada para o modelo é um par (tokens de entrada, tokens de destino), 
    por exemplo, uma frase em inglês (tokenizada) e sua tradução para o 
    alemão (tokenizada)

    Args:
    input_vocab_size: int: tamanho do vocabulário da entrada
    target_vocab_size: int: tamanho do vocabulário do alvo
    d_model: int:  profundidade do (n_units na celula LSTM)
    n_encoder_layers: int: número de camadas LSTM no codificador
    n_decoder_layers: int: número de camadas LSTM no decodificador após atenção
    n_attention_heads: int: número de "heads" de atenção. Número variáveis na pilha de sinais 
    que serão passadas para a camada de atenção. Normalmente é igual a 4 
    [queries, keys, values, mask]
    attention_dropout: float, dropout para a camada de atenção
    mode: str: 'train', 'eval' or 'predict', o modo de previsão é para inferência rápida

    Retornos:
    Um modelo de sequência a sequência LSTM com atenção..
    """
    
    # Passo 0: chamada da função auxiliar input_encoder_fn para para preparar as entradas para o "Input Encoder"
    input_encoder = input_encoder_fn(input_vocab_size, d_model, n_encoder_layers)

    # Passo 0: chamada da função auxiliar pre_attention_decoder_fn para preparar as entradas para o "pre-attention decoder"
    pre_attention_decoder = pre_attention_decoder_fn(mode, target_vocab_size, d_model)

    # Passo 1: Cria uma rede em série. No trax as operações são adicionadas em forma de pilha LIFO (last in, first out)
    model = tl.Serial( 
        
      # Passo 2: armazene tokens de entrada e tokens de destino, pois serão necessários posteriormente.
      tl.Select([0,1,0,1]),
        
      # Passo 3: execute o codificador de entrada na entrada e o decodificador de pré-atenção no alvo.
      tl.Parallel(input_encoder, pre_attention_decoder),
        
      # Passo 4: prepara queries, keys, values and mask for attention.
      tl.Fn('PrepareAttentionInput', prepare_attention_input, n_out=4),
        
      # Passo 5: executa a camada de attentionr
      # aninhe-o dentro de uma camada residual para adicionar às ativações do decodificador de pré-atenção (ou seja, queries)
      tl.Residual(tl.AttentionQKV(d_model, n_heads=n_attention_heads, dropout=attention_dropout, mode=mode)),
      
      # Passo 6: remova a máscara da pilha de sinais da rede
      tl.Select([0,2]),
        
      # Passo 7: executa o restante no decodificador LSTM
      [tl.LSTM(n_units=d_model) for _ in range(n_decoder_layers)],
        
      # Passo 8: prepare output by making it the right size
      tl.Dense(target_vocab_size),
        
      # Passo 9: Aplicase-se Log-softmax na saída
      tl.LogSoftmax()
    )
    
    return model

In [50]:
# print your model
model = NMTAttn()
print(model)

Serial_in2_out2[
  Select[0,1,0,1]_in2_out4
  Parallel_in2_out2[
    Serial[
      Embedding_33300_1024
      LSTM_1024
      LSTM_1024
    ]
    Serial[
      ShiftRight(1)
      Embedding_33300_1024
      LSTM_1024
    ]
  ]
  PrepareAttentionInput_in3_out4
  Serial_in4_out2[
    Branch_in4_out3[
      None
      Serial_in4_out2[
        Parallel_in3_out3[
          Dense_1024
          Dense_1024
          Dense_1024
        ]
        PureAttention_in4_out2
        Dense_1024
      ]
    ]
    Add_in2
  ]
  Select[0,2]_in3_out2
  LSTM_1024
  LSTM_1024
  Dense_33300
  LogSoftmax
]


<a name="3"></a>
# Part 3:  Treinamento

Nessa seção será ralizado o treinamento do mo0delo. Fazer treinamento supervisionado no Trax é bastante simples (pequeno exemplo aqui [here](https://trax-ml.readthedocs.io/en/latest/notebooks/trax_intro.html#Supervised-training)). Para isso será instanciado três classes: `TrainTask`, `EvalTask`, and `Loop`. Vamos dar uma olhada em cada um deles nas seções abaixo.

## 3.1  TrainTask

A classe [TrainTask](https://trax-ml.readthedocs.io/en/latest/trax.supervised.html#trax.supervised.training.TrainTask) nos permite definir os dados rotulados para usar no treinamento e os mecanismos de feedback para calcular a perda e atualizar os pesos.


In [51]:
train_task = training.TrainTask(
    
    # utilização da stream (gerador) batch de treinamento como dados rotulados
    labeled_data= train_batch_stream,
    
    # utilização d-*a perda de entropia cruzada
    loss_layer= tl.CrossEntropyLoss(),
    
    # use o otimizador Adam com taxa de aprendizagem de 0,01
    optimizer= trax.optimizers.Adam(0.01),
    
    # use `trax.lr.warmup_and_rsqrt_decay` para atualizar a taxa de aprendizagem
    lr_schedule= trax.lr.warmup_and_rsqrt_decay(1000, 0.01),
    
    # checkpoint a cada 10 steps
    n_steps_per_checkpoint= 10,
)

<a name="3.2"></a>
## 3.2  EvalTask

The [EvalTask](https://trax-ml.readthedocs.io/en/latest/trax.supervised.html#trax.supervised.training.EvalTask) por outro lado, permite-nos ver como está o desempenho do modelo durante o treino. Nesse trabalho, serão avaliadas a acurácia e a perda de cross entropia.

In [53]:
eval_task = training.EvalTask(
    
    ## utilização da stream (gerador) batch de validação como dados rotulados
    labeled_data=eval_batch_stream,
    
    ## utilização da perda de entropia cruzada e acurácia como métricas.
    metrics=[tl.CrossEntropyLoss(), tl.Accuracy()],
)

<a name="3.3"></a>
## 3.3  Loop

A classe [Loop](https://trax-ml.readthedocs.io/en/latest/trax.supervised.html#trax.supervised.training.Loop) efine o modelo que treinaremos, bem como as tarefas de treinamento e avaliação a serem executadas. Seu método `run ()` nos permite executar o treinamento para um determinado número de etapas.

In [57]:
# define o diretório em que será salvo os pesos do modelo após o treinamento
output_dir = 'output_dir/'

# remova o modelo antigo se ele existir. reinicia o treinamento.
!rm -f ~/output_dir/model.pkl.gz  

# define o loop de treinamento
training_loop = training.Loop(NMTAttn(mode='train'),
                              train_task,
                              eval_tasks=[eval_task],
                              output_dir=output_dir)

In [58]:
# NOTA: Execute o loop de treinamento. Isso levará cerca de 8 minutos para ser concluído.
training_loop.run(10)


Step      1: Ran 1 train steps in 84.14 secs
Step      1: train CrossEntropyLoss |  10.39641666
Step      1: eval  CrossEntropyLoss |  10.37890625
Step      1: eval          Accuracy |  0.00000000

Step     10: Ran 9 train steps in 390.24 secs
Step     10: train CrossEntropyLoss |  10.23944664
Step     10: eval  CrossEntropyLoss |  9.93970108
Step     10: eval          Accuracy |  0.02429765


Observa-se que a a acurácia de validação e o erro de entropia cruzada obtidos não foram satisfatórios, após treinar o modelo por 10 épocas, apesar de ter ocorrido um pequeno aumento na acurácia. Para fazer o teste será utilizado o mesmo modelo, mas que foi treinado por um longo período. Será utilizado um modelo pré-treinado, pois é necessário um grande esforço computacional, que para esta atividade, não é algo factível.

<a name="4"></a>
# Part 4:  Teste

Agora usaremos o modelo que acabamos de treinar de treinar para traduzir frases em inglês para alemão. Vamos implementar isso com duas funções: A primeira permite que você identifique o próximo símbolo (ou seja, token de saída). O segundo se encarrega de combinar toda a string traduzida.

Começaremos carregando primeiro uma cópia pré-treinada, por vária épocas, do modelo que você acabou de codificar.

In [59]:
# intancia o modelo criado em modo de validação
model = NMTAttn(mode='eval')

# inicializa os pesos a partir de um modelo pré-treinado
model.init_from_file("model.pkl.gz", weights_only=True)
model = tl.Accelerate(model)

<a name="4.1"></a>
## 4.1  Decoding

A saída do decodificador é um vetor de probabilidades (LogSoftMax). Existem várias maneiras de interpretar a saída do decodificador e assim,obter o próximo token ao traduzir uma frase. Por exemplo, podemos apenas obter o token mais provável em cada etapa (ou seja, decodificação gananciosa ("greed")) ou obter uma amostra de uma distribuição. Podemos generalizar a implementação dessas duas abordagens usando o método `tl.logsoftmax_sample ()`. Vejamos brevemente sua implementação:


```python
def logsoftmax_sample(log_probs, temperature=1.0):  # pylint: disable=invalid-name
  """Returns a sample from a log-softmax output, with temperature.

  Args:
    log_probs: Logarithms of probabilities (often coming from LogSofmax)
    temperature: For scaling before sampling (1.0 = default, 0.0 = pick argmax)
  """
  # This is equivalent to sampling from a softmax with temperature.
  u = np.random.uniform(low=1e-6, high=1.0 - 1e-6, size=log_probs.shape)
  g = -np.log(-np.log(u))
  return np.argmax(log_probs + g * temperature, axis=-1)
```

As principais coisas a serem tiradas aqui são: 1. ele obtém amostras aleatórias com a mesma forma de sua entrada (ou seja, `log_probs`), e 2. a quantidade de" ruído "adicionado à entrada por essas amostras aleatórias é escalada por um configuração de `temperatura`. Ao defini-lo como `0` apenas tornará a instrução de retorno igual a obter o argmax de` log_probs`. Isso será útil mais tarde.

A seguir será implementado a função `next_symbol()` que recebe `input_tokens` e` cur_output_tokens`, retornando o índice da próxima palavra. 


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Observações</b></font>
</summary>
<p>
<ul>
    <li>To get the next power of two, you can compute <i>2^log_2(token_length + 1)</i> . We add 1 to avoid <i>log(0).</i></li>
    <li>You can use <i>np.ceil()</i> to get the ceiling of a float.</li>
    <li><i>np.log2()</i> will get the logarithm base 2 of a value</li>
    <li><i>int()</i> will cast a value into an integer type</li>
    <li>From the model diagram in part 2, you know that it takes two inputs. You can feed these with this syntax to get the model outputs: <i>model((input1, input2))</i>. It's up to you to determine which variables below to substitute for input1 and input2. Remember also from the diagram that the output has two elements: [log probabilities, target tokens]. You won't need the target tokens so we assigned it to _ below for you. </li>
    <li> The log probabilities output will have the shape: (batch size, decoder length, vocab size). It will contain log probabilities for each token in the <i>cur_output_tokens</i> plus 1 for the start symbol introduced by the ShiftRight in the preattention decoder. For example, if cur_output_tokens is [1, 2, 5], the model will output an array of log probabilities each for tokens 0 (start symbol), 1, 2, and 5. To generate the next symbol, you just want to get the log probabilities associated with the last token (i.e. token 5 at index 3). You can slice the model output at [0, 3, :] to get this. It will be up to you to generalize this for any length of cur_output_tokens </li>
</ul>


In [63]:
def next_symbol(NMTAttn, input_tokens, cur_output_tokens, temperature):
    """Retorna o índice do próximo token

    Args:
        NMTAttn (tl.Serial): Um modelo LSTM sequence-to-sequence com attention 
        input_tokens (np.ndarray 1 x n_tokens): reprezentação tokenizada da sentença de entrada
        cur_output_tokens (list): reprezentação tokenizada da sentença de entrada das palavras traduzidas previamente 
        temperature (float): parametro de ruído que varia entre 0.0 to 1.0.
            0.0: o mesmo que argmax, sempre escolhendo a palavra mais provavel
            1.0: amostra a partir de uma distribuição (pode levar a traduções aleatórias)

    Returns:
        int: index of the next token in the translated sentence
        float: log probability of the next symbol
    """

    # definir o comprimento dos tokens de saída atuais
    token_length = len(cur_output_tokens)

    # calcular a próxima potência de 2 para o comprimento de padding
    padded_length = np.power(2, int(np.ceil(np.log2(token_length + 1))))

    # pad cur_output_tokens até padded_length
    padded = cur_output_tokens + [0] * (padded_length - token_length)
    
    # model expects the output to have an axis for the batch size in front so
    # convert `padded` list to a numpy array with shape (None, <padded_length>) where
    # None is a placeholder for the batch size
    
    # o modelo espera que a saída tenha um eixo para o tamanho do lote na frente, então
    # converter a lista `padded` em uma matriz numpy com forma (Nenhum, <padded_length>) onde
    # None é um espaço reservado para o tamanho do lote
    padded_with_batch = np.expand_dims(padded, axis=0)

    
    # obter a previsão do modelo (lembre-se de usar o argumento `NMAttn` definido acima)
    # get the model prediction (remember to use the `NMAttn` argument defined above)
    output, _ = NMTAttn((input_tokens, padded_with_batch))
    
    # get log probabilities from the last token output
    log_probs = output[0, token_length, :]

    # get the next symbol by getting a logsoftmax sample (*hint: cast to an int)
    symbol = int(tl.logsoftmax_sample(log_probs, temperature))

    return symbol, float(log_probs[symbol])

Agora será implementado a função `sampling_decode ()`. Isso irá chamar a função `next_symbol ()` acima várias vezes até que a próxima saída seja o token de final de frase (ou seja, `EOS`). Ele recebe uma string de entrada e retorna a versão traduzida dessa string.

In [61]:
def sampling_decode(input_sentence, NMTAttn = None, temperature=0.0, vocab_file=None, vocab_dir=None):
    """Returns sentenças traduzidas

    Args:
        input_sentence (str): sentença a ser traduzida
        NMTAttn (tl.Serial): Um modelo LSTM sequence-to-sequence com attention
        temperature (float): parametro de ruído que varia entre 0.0 to 1.0.
            0.0: o mesmo que argmax, sempre escolhendo a palavra mais provavel
            1.0: amostra a partir de uma distribuição (pode levar a traduções aleatórias)
        vocab_file (str): nome do arquivo que contem o vocabularion
        vocab_dir (str): caminho para o arquivo do vocabularion

    Returns:
        tuple: (list, str, float)
            list of int: versão tokenizada da sentença traduzida
            float: log probability da frase traduzida
            str: a sentença traduzida
    """
    
    # codifica a sentença de entrada
    input_tokens = tokenize(input_sentence,vocab_file,vocab_dir)
    
    # inicializa a lista de tokens de saída
    cur_output_tokens = []
    
    # initialize um inteiro que representa o índice atual de saísa
    cur_output = 0
    
    # Define o token de "fim de sentença" como 1
    EOS = 1
    
    # enquanto a saída atual não for o fim de sequência
    while cur_output != EOS:
        
        # atualize o token de saída atual obtendo o índice da próxima palavra
        cur_output, log_prob = next_symbol(NMTAttn, input_tokens, cur_output_tokens, temperature)
        
        # anexar o token de saída atual à lista de tokens de saída
        cur_output_tokens.append(cur_output)
    
    # detokenize os tokens de saída
    sentence = detokenize(cur_output_tokens, vocab_file, vocab_dir)
    
    return cur_output_tokens, log_prob, sentence

In [64]:
# Test the function above. Try varying the temperature setting with values from 0 to 1.
# Run it several times with each setting and see how often the output changes.
sampling_decode("I love languages.", model, temperature=0.0, vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR)

([161, 12202, 5112, 3, 1], -0.0001735687255859375, 'Ich liebe Sprachen.')

We have set a default value of `0` to the temperature setting in our implementation of `sampling_decode()` above. As you may have noticed in the `logsoftmax_sample()` method, this setting will ultimately result in greedy decoding. As mentioned in the lectures, this algorithm generates the translation by getting the most probable word at each step. It gets the argmax of the output array of your model and then returns that index. See the testing function and sample inputs below. You'll notice that the output will remain the same each time you run it.

In [73]:
import sacrebleu  # import sacrebleu in order compute the BLEU score.

eval_batch, target_ebal_batch, mask_eval_batch = next(eval_batch_stream)

# escolhe um índice aleatório menor que o tamanho do batch.
Bleu = 0

i = 0

# Calculo do BLEU sobre um batch dos dados de validação
for i in range(len(eval_batch)):
    
    input_sentence = detokenize(eval_batch[i], vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR)
    _,_, translated_sentenc = sampling_decode(input_sentence, model, temperature=0.0, vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR)

    Bleu += sacrebleu.corpus_bleu(translated_sentenc, input_sentence).score


meanBLEU = Bleu/(i+1)

64


In [79]:
print(round(meanBLEU,1))

26.5


**Interpretação do BLEU score sobre um corpus**

|Score      | Interpretation                                                |
|:---------:|:-------------------------------------------------------------:|
| < 10      | Almost useless                                                |
| 10 - 19   | Hard to get the gist                                          |
| 20 - 29   | The gist is clear, but has significant grammatical errors     |
| 30 - 40   | Understandable to good translations                           |
| 40 - 50   | High quality translations                                     |
| 50 - 60   | Very high quality, adequate, and fluent translations          |
| > 60      | Quality often better than human                               |

Não foi calculado o BLEU score sobre o corpus, mas apenas sobre um batch dos dados de validação, pois a função `sampling_decode` é custosa. Mas os dados do batch são aleatórios dando um indício do BLEU score sobre o corpus. De acordo com a tabela acima (https://cloud.google.com/translate/automl/docs/evaluate) o BleuScore médio obtido indica que foram obtidas traduções é claras, mas que contém erros gramaticais significativos.

Foi executado a busca gulosa, entretanto obter o token mais provável em cada etapa pode não produzir necessariamente os melhores resultados. provavelmente seriam obtidos melhores resultados se fosse utilizada outra abordagem, como o Beam Search.
